# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from PositionalModel import PositionalModel

from src.match_names import *
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(FEATURES_OUTFIELD)

47

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape

(34514, 274)

In [7]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [8]:
# df = df[df["GW"] == 2]

In [9]:
# X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]
df = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [10]:
df.isna().sum().sort_values(ascending=False).head(20)

Min_points_2     16744
Min_points       16744
FPL_GW           16744
Clean_name       16744
Min_points_30    16744
Min_points_4     16744
Cmp%             15960
Carries          15954
PrgP             15954
Blocks           15954
Att              15954
PrgC             15954
Cmp              15954
GCA              15954
SCA              15954
xA               15954
T_att            15954
npxG             15954
T_succ           15954
xG               15954
dtype: int64

In [11]:
# df[FEATURES_OUTFIELD].iloc[0]

In [12]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [13]:
df.shape

(16744, 274)

In [14]:
X_test = df.copy()

In [15]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [16]:
file = open("models/GBR_positional.pkl",'rb')
model = pickle.load(file)

In [17]:
model

In [18]:
model.predict( X_test.head(5) )

[0.1480629526080559,
 0.4321149964239961,
 3.2790989413109752,
 2.4608015540791177,
 3.104018975431684]

In [19]:
model.predict( X_test.head(3) )

[0.1480629526080559, 0.4321149964239961, 3.2790989413109752]

In [20]:
model.predict( X_test.head(4) )

[0.1480629526080559,
 0.4321149964239961,
 3.2790989413109752,
 2.4608015540791177]

In [21]:
X_test.head(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
107,107,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117318,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True
108,108,2023-08-26,Sat,Home,Brentford,Crystal Palace,Aaron-Hickey,1.0,RB,90.0,...,0.69,1774.125598,1769.467049,NaN,1.466667,1.0,0.0,0.0,0.0,True


In [22]:
X_test.head(3)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True


# Predicting

In [23]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [24]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [25]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [26]:
X_test[FEATURES_OUTFIELD].isna()

,Was_home,Rating_difference,Price,Transfers_balance,Avg_FPL_points,Min_2,Gls_2,Sh_2,SoT_2,Ast_2,...,Ast_30,xG_30,xA_30,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,Player_GC_30,FPL_points_30,xP_30
76,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
77,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
107,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
108,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34509,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34510,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34511,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34512,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
# X_test.shape,  model.predict(X_test).shape

In [28]:
X_test["Pos"].isna().sum()

0

In [29]:
%%time
preds = get_predictions(model, df, X_test)

16744 (16744, 274)
CPU times: total: 594 ms
Wall time: 300 ms


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [30]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
12105,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,12.5,5.705067
13363,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,7.6,5.626141
5618,Ethan Pinnock,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,DEF,4.5,5.619551
13852,Rico Henry,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,DEF,4.6,5.390172
16082,Virgil van Dijk,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,6.0,5.109740
15313,Thiago Emiliano da Silva,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,DEF,5.0,5.081692
1883,Andrew Robertson,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,6.5,5.015730
13502,Raheem Sterling,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,MID,7.1,5.005739
3034,Bukayo Saka,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,MID,8.7,4.981443
15802,Trent Alexander-Arnold,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,7.9,4.977333


In [31]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
1498,Alisson Ramses Becker,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,GK,5.5,4.593270
6316,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,GK,5.0,4.558261
13989,Robert Sánchez,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,GK,4.5,3.975769
5026,Ederson Santana de Moraes,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,GK,5.5,3.972971
16291,Wes Foderingham,4.0,2023-24,Sheffield United,Everton,1.0,1631.764282,1691.463989,GK,4.5,3.774245
8937,José Malheiro de Sá,4.0,2023-24,Wolves,Crystal Palace,0.0,1704.537109,1758.725586,GK,5.0,3.745701
1008,Aaron Ramsdale,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,GK,5.0,3.425215
12628,Nick Pope,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,GK,5.5,3.319186
1603,Alphonse Areola,4.0,2023-24,West Ham,Luton,0.0,1787.825562,1602.477417,GK,4.0,3.237212
14337,Sam Johnstone,4.0,2023-24,Crystal Palace,Wolves,1.0,1758.725586,1704.537109,GK,4.5,3.199316


In [32]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
12111,Mohamed Salah,10.0,2023-24,Liverpool,Nott'ham Forest,1.0,1946.521606,1677.877930,MID,12.5,7.035485
12127,Mohamed Salah,26.0,2023-24,Liverpool,Luton,1.0,1946.521606,1602.477417,MID,12.5,6.999492
12132,Mohamed Salah,31.0,2023-24,Liverpool,Sheffield United,1.0,1946.521606,1631.764282,MID,12.5,6.921971
12110,Mohamed Salah,9.0,2023-24,Liverpool,Everton,1.0,1946.521606,1691.463989,MID,12.5,6.711881
832,Thiago Emiliano da Silva,3.0,2023-24,Chelsea,Luton Town,1.0,1781.043579,1602.477417,DEF,5.0,6.522731
122,Bruno Borges Fernandes,2.0,2023-24,Manchester Utd,Tottenham,0.0,1877.037720,1817.656738,MID,8.5,6.508540
12115,Mohamed Salah,14.0,2023-24,Liverpool,Fulham,1.0,1946.521606,1732.639404,MID,12.5,6.462424
12139,Mohamed Salah,38.0,2023-24,Liverpool,Wolves,1.0,1946.521606,1704.537109,MID,12.5,6.455006
13391,Phil Foden,33.0,2023-24,Manchester City,Luton,1.0,2083.702148,1602.477417,MID,7.6,6.453595
12125,Mohamed Salah,24.0,2023-24,Liverpool,Burnley,1.0,1946.521606,1722.952881,MID,12.5,6.435034


In [33]:
preds[preds["FPL_pos"] == "GK"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
8959,José Malheiro de Sá,26.0,2023-24,Wolves,Sheffield United,1.0,1704.537109,1631.764282,GK,5.0,5.803838
35,Alisson Ramses Becker,1.0,2023-24,Liverpool,Chelsea,0.0,1946.899780,1788.094360,GK,5.5,5.689138
6342,Guglielmo Vicario,30.0,2023-24,Tottenham,Luton,1.0,1829.239014,1602.477417,GK,5.0,5.549021
6337,Guglielmo Vicario,25.0,2023-24,Tottenham,Wolves,1.0,1829.239014,1704.537109,GK,5.0,5.495897
6317,Guglielmo Vicario,5.0,2023-24,Tottenham,Sheffield United,1.0,1829.239014,1631.764282,GK,5.0,5.495109
8968,José Malheiro de Sá,35.0,2023-24,Wolves,Luton,1.0,1704.537109,1602.477417,GK,5.0,5.414263
6321,Guglielmo Vicario,9.0,2023-24,Tottenham,Fulham,1.0,1829.239014,1732.639404,GK,5.0,5.277056
6332,Guglielmo Vicario,20.0,2023-24,Tottenham,Bournemouth,1.0,1829.239014,1660.290894,GK,5.0,5.249771
6344,Guglielmo Vicario,32.0,2023-24,Tottenham,Nott'ham Forest,1.0,1829.239014,1677.877930,GK,5.0,5.232689
6339,Guglielmo Vicario,27.0,2023-24,Tottenham,Crystal Palace,1.0,1829.239014,1758.725586,GK,5.0,5.215364


In [34]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Mohamed Salah,210.553268
Bruno Borges Fernandes,195.471152
Bukayo Saka,193.826005
Phil Foden,193.304171
Luis Díaz,177.977867
Andrew Robertson,176.687933
Guglielmo Vicario,176.596591
Virgil van Dijk,175.458761
Lyle Foster,174.082604


# Saving to file

In [35]:
preds.to_csv("predictions/preds_next_season.csv", index=False)